In [23]:
# test xml parser with python
import xml.etree.ElementTree as ET
tree = ET.parse('/Users/wshuo/Desktop/data4KG/packages/source/velodyne_simulator/velodyne_description/package.xml')
root = tree.getroot()
root.find('name').text


'velodyne_description'

In [25]:
import os
import sys
import xml.etree.ElementTree as ET

# traverse the source folder, analyze each package

source_path = "/Users/wshuo/Desktop/data4KG/packages/source"

cmakelists_count = 0
packagexml_count = 0



def is_meta_pkg(path):
    
    #check the content of CMakeLists.txt "catkin_metapackage()"
    cmake_file = os.path.join(path, "CMakeLists.txt")
    with open(cmake_file) as myfile:
        if 'catkin_metapackage()' in myfile.read() and ('#catkin_metapackage()' not in myfile.read()) and ('# catkin_metapackage()' not in myfile.read()):
            return True
    return False
    
def get_pkg_name(path):
    package_xml_file = os.path.join(path, "package.xml")
    
    tree = ET.parse(package_xml_file)
    root = tree.getroot()
    name = root.find('name').text
    return name.strip()
     
    
def get_pkg_type(path):
    
    #check the content of CMakeLists.txt "catkin_metapackage()"
    cmake_file = os.path.join(path, "CMakeLists.txt")
    package_name = get_pkg_name(path)
    
    
    with open(cmake_file) as myfile:
        if 'catkin_metapackage()' in myfile.read() and ('#catkin_metapackage()' not in myfile.read()) and ('# catkin_metapackage()' not in myfile.read()):
            return package_name,'meta_package'
    
    

    suffix = package_name.split('_')[-1]
    
    has_msg_folder = False
    has_mesh_folder = False
    for root, dirs, files in os.walk(path, topdown=True):
        if 'msg' in dirs:
            has_msg_folder = True
        if 'meshes' in dirs:
            has_mesh_folder = True       
        
        break
    
    
    
    if suffix == "msgs" and has_msg_folder:
        return package_name,'message_package'
    
    if suffix == "description" and has_mesh_folder:
        return package_name,'description_package'
    
    
    
    return package_name,'function_package'



    

In [26]:
# get_pkg_type
get_pkg_type("/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/3rdparty/mini_maxwell")

('mini_maxwell', 'function_package')

In [27]:
import csv

def traverse_pkgs(path):
    package_count = 0
    packagexml_count = 0

    pkg_path_file = open('../output/package_path.txt', 'w', encoding='utf-8')

    
    
    #write relations to files
    pkg_type_csv = open('../output/package_type.csv', 'w', encoding='utf-8')
    pkg_type_header = ['packageName','type']
    pkg_type_writer = csv.DictWriter(pkg_type_csv, pkg_type_header)
    pkg_type_writer.writeheader()
    

    
    for root, dirs, files in os.walk(path, topdown=True):
        print(len(dirs))
        for name in dirs:
            # print("∏",os.path.join(root, name))
            repo_path = os.path.join(root, name)
            for repo_root, repo_dirs, repo_files in os.walk(repo_path, topdown=True):
                # if only one package, then the repo name is just the package name



                # check cmakelist whether in repo
                # for repo_file in repo_files:
                is_package = False
                if ("package.xml" in repo_files) and "CMakeLists.txt" in repo_files:
    #                 cmakelists_count += 1
                    is_package = True
                if ("package.xml" in repo_files):
                    packagexml_count += 1
                
                
                paths = []
                if is_package:
                    paths.append(repo_path)
                        
                    package_count += 1

                if not is_package:
                    for repo_dir in repo_dirs:
                        package_path = os.path.join(repo_path, repo_dir)
                        for pkg_root, pkg_dirs, pkg_files in os.walk(package_path, topdown=True):
                            if ("package.xml" in pkg_files) and "CMakeLists.txt" in pkg_files:   # force two files, as in 
                                # /Users/wshuo/Desktop/data4KG/packages/source/md49_base_controller/src/CMakeLists.txt'  not exist
                                # is a package
#                                 print(is_meta_pkg(package_path))
                                paths.append(package_path)
                                package_count += 1
                            if ("package.xml" in pkg_files):
                                packagexml_count += 1
                            for pkg_dir in pkg_dirs:
                                new_package_path = os.path.join(package_path, pkg_dir)
                                for new_pkg_root, new_pkg_dirs, new_pkg_files in os.walk(new_package_path, topdown=True):
                                    if ("package.xml" in new_pkg_files) and "CMakeLists.txt" in new_pkg_files:   # force two files, as in 
                                        # /Users/wshuo/Desktop/data4KG/packages/source/md49_base_controller/src/CMakeLists.txt'  not exist
                                        # is a package
#                                         print(is_meta_pkg(new_package_path))
                                        paths.append(new_package_path)
                                        package_count += 1
                                    if ("package.xml" in new_pkg_files):
                                        packagexml_count += 1
                                    break



                            break

                            
                for path_pkg in paths:
                
                    pkg_path_file.write(path_pkg+'\n')
                    
                
                    package_name, package_type = get_pkg_type(path_pkg)
                    if package_type:
                        data_type = {'packageName':package_name,'type':package_type}
                        pkg_type_writer.writerow(data_type)
                break
        break



    print(package_count)
    print(packagexml_count)
    pkg_path_file.close()
    pkg_type_csv.close()

In [28]:
is_meta_pkg("/Users/wshuo/Desktop/data4KG/packages/source/fsrobo_r/fsrobo_r")






False

In [29]:
traverse_pkgs(source_path)







1140
3518
3518


In [30]:
import os
import re

def parse_cmakelist(path_to_ros_pkg):
    package_path = path_to_ros_pkg
    
#     if is_meta_pkg(package_path):
#         return False
    
    
    #todo:  python scripts node .   
    
#     package_path = os.path.join(os.environ['ROS_PACKAGE_PATH'].split(':')[0], package_name)
    
    cmake_file_path = os.path.join(package_path, "CMakeLists.txt")
    cmake_file = open(cmake_file_path, "r")
    cmake_data = cmake_file.read()

    executable_regex = r"add_executable\((\S+) (\S+.cpp|\S+.c|\S+.cxx|\S+.cc|\S+.c\+\+|)"
    executable_matches = re.findall(executable_regex, cmake_data)

    node_files = []
    for match in executable_matches:
        executable_name = match[0]
        if executable_name != "${PROJECT_NAME}_node":
            if executable_name == "${PROJECT_NAME}":
                package_name = get_pkg_name(path_to_ros_pkg)
                node_files.append(package_name)
            else:
                node_files.append(executable_name)
    
    # python node
    script_folder = os.path.join(package_path, "scripts")
    if os.path.isdir(script_folder):
        node_files = [f for f in os.listdir(script_folder) if f.endswith(".py")]
        


    
    # message
    msg_folder = os.path.join(package_path, "msg")
    msg_files = []
#     print(os.listdir(msg_folder))
    if os.path.isdir(msg_folder):
        msg_files = [f for f in os.listdir(msg_folder) if f.endswith(".msg")]
        
#     print(msg_files)


    # service
    srv_folder = os.path.join(package_path, "srv")
    srv_files = []
    if os.path.isdir(srv_folder):
        srv_files = [f for f in os.listdir(srv_folder) if f.endswith(".srv")]
        
#     print(srv_files)
    
    
    
    # action
    action_folder = os.path.join(package_path, "action")
    action_files = []
    if os.path.isdir(action_folder):
        action_files = [f for f in os.listdir(action_folder) if f.endswith(".action")]
        
#     print(action_files)
    
    
    # launch
    launch_folder = os.path.join(package_path, "launch")
    launch_files = []
    if os.path.isdir(launch_folder):
        launch_files = [f for f in os.listdir(launch_folder) if f.endswith(".launch")]
        
#     print(launch_files)
    
    result = dict()
    result['node'] = node_files
    result['msg'] = msg_files
    result['srv'] = srv_files
    result['action'] = action_files
    result['launch'] = launch_files
    
    
    
    return result
    

In [31]:
# parse_cmakelist("/Users/wshuo/Desktop/data4KG/packages/source/fsrobo_r/fsrobo_r")
# parse_cmakelist("/Users/wshuo/Desktop/data4KG/packages/source/raspicat_ros")
parse_cmakelist("/Users/wshuo/Desktop/data4KG/packages/source/robosense/rslidar_msgs")
parse_cmakelist("/Users/wshuo/Desktop/data4KG/packages/source/move_base_sequence")
parse_cmakelist("/Users/wshuo/Desktop/data4KG/packages/source/asr_direct_search_manager")

{'node': [],
 'msg': [],
 'srv': [],
 'action': ['directSearch.action'],
 'launch': ['direct_search_manager.launch']}

In [32]:
def test_parse_cmakelist(package_path_file):
    
    file = open(package_path_file, 'r', encoding='utf-8')
    
    pkg_launch_csv = open('../output/package_launch.csv', 'w', encoding='utf-8')
    pkg_launch_header = ['packageName','launch']
    pkg_launch_writer = csv.DictWriter(pkg_launch_csv, pkg_launch_header)
    pkg_launch_writer.writeheader()
    
    pkg_node_csv = open('../output/package_node.csv', 'w', encoding='utf-8')
    pkg_node_header = ['packageName','node']
    pkg_node_writer = csv.DictWriter(pkg_node_csv, pkg_node_header)
    pkg_node_writer.writeheader()
    
    pkg_srv_csv = open('../output/package_srv.csv', 'w', encoding='utf-8')
    pkg_srv_header = ['packageName','srv']
    pkg_srv_writer = csv.DictWriter(pkg_srv_csv, pkg_srv_header)
    pkg_srv_writer.writeheader()

    pkg_action_csv = open('../output/package_action.csv', 'w', encoding='utf-8')
    pkg_action_header = ['packageName','action']
    pkg_action_writer = csv.DictWriter(pkg_action_csv, pkg_action_header)
    pkg_action_writer.writeheader()

    pkg_msg_csv = open('../output/package_msg.csv', 'w', encoding='utf-8')
    pkg_msg_header = ['packageName','msg']
    pkg_msg_writer = csv.DictWriter(pkg_msg_csv, pkg_msg_header)
    pkg_msg_writer.writeheader()
    

    
#     pkg_type_writer.writerows(data)
    
    

    

    for line in file:
        path = line.strip("\n")
        print(path)
        # todo
        package_name = get_pkg_name(path)
        
        
        if is_meta_pkg(path):
            continue
        result = parse_cmakelist(path)
        for node in result['node']:
            data_row = {'packageName':package_name, 'node':node}
            pkg_node_writer.writerow(data_row)
        for msg in result['msg']:
            data_row = {'packageName':package_name, 'msg':msg}
            pkg_msg_writer.writerow(data_row)
        for action in result['action']:
            data_row = {'packageName':package_name, 'action':action}
            pkg_action_writer.writerow(data_row)
        for launch in result['launch']:
            data_row = {'packageName':package_name, 'launch':launch}
            pkg_launch_writer.writerow(data_row)
        for srv in result['srv']:
            data_row = {'packageName':package_name, 'srv':srv}
            pkg_srv_writer.writerow(data_row)
        
        
    pkg_launch_csv.close()
    pkg_srv_csv.close()
    pkg_action_csv.close()
    pkg_msg_csv.close()
    pkg_node_csv.close()
        


        
        

test_parse_cmakelist("../output/package_path.txt")







/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_navigation
/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_map
/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_rawlog
/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_tutorials
/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_local_obstacles
/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_localization
/Users/wshuo/Desktop/data4KG/packages/source/mrpt_navigation/mrpt_reactivenav2d
/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/jsk_3rdparty
/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/switchbot_ros
/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/rostwitter
/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/chaplus_ros
/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/dialogflow_task_executive
/Users/wshuo/Desktop/data4KG/packages/source/jsk_3rdparty/google_chat_ros
/Users/wshuo/Desktop/da

/Users/wshuo/Desktop/data4KG/packages/source/baldor
/Users/wshuo/Desktop/data4KG/packages/source/nmea_gps_plugin
/Users/wshuo/Desktop/data4KG/packages/source/flir_camera_driver/flir_camera_driver
/Users/wshuo/Desktop/data4KG/packages/source/flir_camera_driver/spinnaker_camera_driver
/Users/wshuo/Desktop/data4KG/packages/source/flir_camera_driver/flir_camera_description
/Users/wshuo/Desktop/data4KG/packages/source/web_video_server
/Users/wshuo/Desktop/data4KG/packages/source/lslidar_C16/lslidar_c16
/Users/wshuo/Desktop/data4KG/packages/source/lslidar_C16/lslidar_c16_decoder
/Users/wshuo/Desktop/data4KG/packages/source/lslidar_C16/lslidar_c16_compensator
/Users/wshuo/Desktop/data4KG/packages/source/lslidar_C16/lslidar_c16_driver
/Users/wshuo/Desktop/data4KG/packages/source/lslidar_C16/lslidar_c16_msgs
/Users/wshuo/Desktop/data4KG/packages/source/turtlebot3_autorace/turtlebot3_autorace_detect
/Users/wshuo/Desktop/data4KG/packages/source/turtlebot3_autorace/turtlebot3_autorace
/Users/wshuo

/Users/wshuo/Desktop/data4KG/packages/source/depthcloud_encoder
/Users/wshuo/Desktop/data4KG/packages/source/asr_ros_uri
/Users/wshuo/Desktop/data4KG/packages/source/hector_nist_arenas_gazebo/hector_nist_arena_worlds
/Users/wshuo/Desktop/data4KG/packages/source/hector_nist_arenas_gazebo/hector_nist_arena_designer
/Users/wshuo/Desktop/data4KG/packages/source/hector_nist_arenas_gazebo/hector_nist_arena_elements
/Users/wshuo/Desktop/data4KG/packages/source/agvs_common/agvs_common
/Users/wshuo/Desktop/data4KG/packages/source/agvs_common/agvs_pad
/Users/wshuo/Desktop/data4KG/packages/source/agvs_common/agvs_description
/Users/wshuo/Desktop/data4KG/packages/source/unique_identifier/unique_id
/Users/wshuo/Desktop/data4KG/packages/source/unique_identifier/unique_identifier
/Users/wshuo/Desktop/data4KG/packages/source/unique_identifier/uuid_msgs
/Users/wshuo/Desktop/data4KG/packages/source/cloudwatch-common/cloudwatch_logs_common
/Users/wshuo/Desktop/data4KG/packages/source/cloudwatch-common/da

/Users/wshuo/Desktop/data4KG/packages/source/open_manipulator_msgs
/Users/wshuo/Desktop/data4KG/packages/source/asr_resources_for_vision
/Users/wshuo/Desktop/data4KG/packages/source/rrt_exploration
/Users/wshuo/Desktop/data4KG/packages/source/jackal_desktop/jackal_viz
/Users/wshuo/Desktop/data4KG/packages/source/jackal_desktop/jackal_desktop
/Users/wshuo/Desktop/data4KG/packages/source/rr_swiftnav_piksi
/Users/wshuo/Desktop/data4KG/packages/source/rt_usb_9axisimu_driver
/Users/wshuo/Desktop/data4KG/packages/source/dynamic_reconfigure
/Users/wshuo/Desktop/data4KG/packages/source/rosjava_bootstrap
/Users/wshuo/Desktop/data4KG/packages/source/cob_common/cob_srvs
/Users/wshuo/Desktop/data4KG/packages/source/cob_common/cob_msgs
/Users/wshuo/Desktop/data4KG/packages/source/cob_common/cob_description
/Users/wshuo/Desktop/data4KG/packages/source/cob_common/cob_common
/Users/wshuo/Desktop/data4KG/packages/source/cob_common/raw_description
/Users/wshuo/Desktop/data4KG/packages/source/cob_common/

/Users/wshuo/Desktop/data4KG/packages/source/staubli_experimental/staubli_experimental
/Users/wshuo/Desktop/data4KG/packages/source/staubli_experimental/staubli_rx160_gazebo
/Users/wshuo/Desktop/data4KG/packages/source/staubli_experimental/staubli_tx90_support
/Users/wshuo/Desktop/data4KG/packages/source/staubli_experimental/staubli_tx60_gazebo
/Users/wshuo/Desktop/data4KG/packages/source/asr_mild_base_launch_files
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device_msgs
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device_utils
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device_bringup
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device_srvs
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device_driver
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_device_description
/Users/wshuo/Desktop/data4KG/packages/source/qbdevice-ros/qb_d

/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/test/test_rosmaster
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/test/test_rosservice
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/test/test_rosbag_storage
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/test/test_rospy
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/test/test_roslib_comm
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/utilities/message_filters
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/utilities/roswtf
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/utilities/xmlrpcpp
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/utilities/roslz4
/Users/wshuo/Desktop/data4KG/packages/source/ros_comm/ros_comm
/Users/wshuo/Desktop/data4KG/packages/source/find_moving_objects
/Users/wshuo/Desktop/data4KG/packages/source/cob_driver/cob_bms_driver
/Users/wshuo/Desktop/data4KG/packages/source/cob_driver/cob_canopen_motor
/Users/wshuo/Desktop/data4KG/packages/source/cob_driver/co

/Users/wshuo/Desktop/data4KG/packages/source/urg_node
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/force_torque_sensor_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/gripper_action_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/imu_sensor_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/four_wheel_steering_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/diff_drive_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/joint_trajectory_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/rqt_joint_trajectory_controller
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/position_controllers
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/effort_controllers
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/velocity_controllers
/Users/wshuo/Desktop/data4KG/packages/source/ros_controllers/ros_controllers
/Users

/Users/wshuo/Desktop/data4KG/packages/source/sicktoolbox_wrapper
/Users/wshuo/Desktop/data4KG/packages/source/create_autonomy/ca_tools
/Users/wshuo/Desktop/data4KG/packages/source/create_autonomy/ca_driver
/Users/wshuo/Desktop/data4KG/packages/source/create_autonomy/ca_msgs
/Users/wshuo/Desktop/data4KG/packages/source/create_autonomy/create_autonomy
/Users/wshuo/Desktop/data4KG/packages/source/create_autonomy/ca_description
/Users/wshuo/Desktop/data4KG/packages/source/odom_frame_publisher
/Users/wshuo/Desktop/data4KG/packages/source/hector_navigation/hector_exploration_controller
/Users/wshuo/Desktop/data4KG/packages/source/hector_navigation/hector_elevation_visualization
/Users/wshuo/Desktop/data4KG/packages/source/hector_navigation/hector_path_follower
/Users/wshuo/Desktop/data4KG/packages/source/hector_navigation/hector_costmap
/Users/wshuo/Desktop/data4KG/packages/source/hector_navigation/hector_gps_calibration
/Users/wshuo/Desktop/data4KG/packages/source/hector_navigation/hector_d